# 🎬 Movie Recommendation System

## Overview
Content-based movie recommender using text similarity
extracted from genres, cast, keywords and overview.

## Dataset
TMDB 5000 Movies + Credits dataset  
Source: Kaggle  
(Not included in repo due to large file size)

## What this notebook does
- Load and preprocess movie metadata
- Create combined text “soup” feature
- Apply TF-IDF Vectorizer
- Compute cosine similarity between movies
- Build a simple recommend(title) function


In [1]:
!mkdir -p src
!mkdir -p model_artifacts


In [1]:
from src.recommender import MovieRecommender

r = MovieRecommender()
r.fit_from_raw("tmdb_5000_movies.csv", "tmdb_5000_credits.csv")
r.load()

print(r.recommend("Mission Impossible"))
print(r.recommend("Avatar"))


[('Mission: Impossible II', 0.2882886514884029), ('Mission: Impossible III', 0.2163327359733048), ('Dying of the Light', 0.19626779175444056), ('Once Upon a Time in Mexico', 0.19370833937565096), ('Bad Company', 0.1773263740708233)]
[('Aliens', 0.1537389684582003), ('Battle: Los Angeles', 0.1422734872557375), ('Falcon Rising', 0.13496626516167365), ('Apollo 18', 0.1314680872034753), ('Star Trek Into Darkness', 0.12661581297398927)]


In [ ]:
import pandas as pd
import numpy as np
import ast

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from difflib import get_close_matches


In [ ]:
# ---------- Load & merge ----------
movies = pd.read_csv("tmdb_5000_movies.csv")
credits = pd.read_csv(
    "tmdb_5000_credits.csv",
    engine="python",          # more tolerant parser
    on_bad_lines="skip"       # skips malformed rows (like 355)
)


In [ ]:
# For your current CSVs, merging on title is easiest (even though id is cleaner)
movies = movies.merge(credits, on="title")

# ---------- Keep needed cols ----------
movies = movies[["movie_id", "title", "overview", "genres", "keywords", "cast", "crew"]]
movies.dropna(inplace=True)


In [ ]:
def parse_names(obj_str, max_items=None, job_filter=None):
    result = []
    for d in ast.literal_eval(obj_str):
        if job_filter and d.get("job") != job_filter:
            continue
        result.append(d["name"])
        if max_items and len(result) >= max_items:
            break
    return [x.replace(" ", "") for x in result]


In [ ]:
def make_soup(row):
    overview = row["overview"] if isinstance(row["overview"], str) else ""
    genres   = " ".join(parse_names(row["genres"]))            # all genres
    keywords = " ".join(parse_names(row["keywords"]))          # all keywords
    cast     = " ".join(parse_names(row["cast"], max_items=3)) # top 3 cast
    crew     = " ".join(parse_names(row["crew"], job_filter="Director"))
    return " ".join([overview, genres, keywords, cast, crew])


## Model Building
Using TF-IDF (better than Bag-of-Words) + Cosine Similarity


In [ ]:
movies["soup"] = movies.apply(make_soup, axis=1)

tfidf = TfidfVectorizer(stop_words="english", max_features=10000)
tfidf_matrix = tfidf.fit_transform(movies["soup"])
similarity = cosine_similarity(tfidf_matrix)

# Map title -> index (case-insensitive)
title_to_index = {t.lower(): i for i, t in enumerate(movies["title"])}


## Recommender Function
Given a movie title, find most similar movies based on metadata


In [ ]:
def recommend(title, top_n=5):
    title = title.lower()

    # if exact match not found, try fuzzy match
    if title not in title_to_index:
        matches = get_close_matches(title, title_to_index.keys(), n=1, cutoff=0.6)
        if not matches:
            return []
        title = matches[0]  # use closest match

    idx = title_to_index[title]
    distances = list(enumerate(similarity[idx]))
    distances = sorted(distances, key=lambda x: x[1], reverse=True)[1 : top_n + 1]
    return [movies.iloc[i[0]].title for i in distances]


## Example Test


In [ ]:
print(recommend("Mission Impossible"))

['Mission: Impossible II', 'Mission: Impossible III', 'Once Upon a Time in Mexico', 'Bad Company', 'Central Intelligence']


In [ ]:
print(recommend("Mission: Impossible"))

['Mission: Impossible II', 'Mission: Impossible III', 'Once Upon a Time in Mexico', 'Bad Company', 'Central Intelligence']


In [ ]:
print(recommend("Avatar"))

['Battle: Los Angeles', 'Star Trek Into Darkness', 'The Book of Life', 'Titan A.E.', 'Alien³']
